In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
sys.path.insert(0, '../')
from src.CCLE_postp_function import *
sys.path.insert(0, '../../JKBio/')
import Datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
from IPython.display import Image,display
tc = TaigaClient()
import numpy as np
from Bio import AlignIO

In [ ]:
# namespace1="broad-genomics-delivery"
# workspace1="Getz_IBM_CellLines_Exomes"
# namespace2="broad-firecloud-ccle"
# workspace2="CCLE_DepMap_WES"
# namespace3="broad-genomics-delivery"
# workspace3="CCL_DepMap_WES"
# refnamespace="broad-firecloud-ccle"
# refworkspace="DepMap_Mutation_Calling_CGA_pipeline"
# source1="ibm"
# source2="ccle"
# source3="ccle"


# samplesetname = "19Q3"
# release = samplesetname

In [ ]:
# newsamples = createDatasetWithNewCellLines(refwm, samplesetname, wm3, source3)


refwm = dm.WorkspaceManager('nci-mimoun-bi-org/CCLF_WES')

# Run the pipeline

In [ ]:
CGAmutations = refwm.get_config("CGA_WES_CCLE_Characterization_Pipeline_v0.1_Jul2019")
CGAmutations

In [ ]:
refwm.update_config(CGAmutations)
submission_id = refwm.create_submission(CGAmutations['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
terra.waitForSubmission(refwm, submission_id)

In [ ]:
filtercommonvar = refwm.get_config("common_variant_filter")
filtercommonvar

In [ ]:
refwm.update_config(filtercommonvar)
submission_id = refwm.create_submission(filtercommonvar['name'], samplesetname, 'sample_set', expression='this.samples')

In [ ]:
terra.waitForSubmission(refwm, submission_id)

In [ ]:
filtermaf = refwm.get_config("filterMAF_on_CGA_pipeline")
filtermaf

In [ ]:
refwm.update_config(filtermaf)
submission_id = refwm.create_submission(filtermaf['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
terra.waitForSubmission(refwm, submission_id)

In [ ]:
# we use this function to merge 19Q3add in 19Q3
terra.updateAllSampleSet(refwm,'19Q3interim', Allsample_setname='19Q3')

In [ ]:
aggregate = refwm.get_config("aggregateMAFs_selectFields_copy")
aggregate

### unfiltered

In [ ]:
samplesetname = '19Q3'

In [ ]:
aggregate['outputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.mergedMAFfn'] = 'this.unfiltered_CGA_MAF_aggregated'
aggregate['inputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.inMafFNs'] = 'this.samples.outMAFannotatedFN'
aggregate['inputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.SelectFields'] = '"Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Genome_Change,Annotation_Transcript,Tumor_Sample_Barcode,cDNA_Change,Codon_Change,Protein_Change,t_alt_count,t_ref_count,tumor_f,Passed_Filters,failure_reasons"'
refwm.update_config(aggregate)
submission_id2 = refwm.create_submission(aggregate['name'], samplesetname)

### filtered

In [ ]:
aggregate['outputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.mergedMAFfn'] = 'this.filtered_CGA_MAF_aggregated'
aggregate['inputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.inMafFNs'] = 'this.samples.outMAFfn'
aggregate['inputs']['aggregateMAFs_selectFields_workflow.aggregateMAFs_selectFields.SelectFields'] = '"Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Genome_Change,Annotation_Transcript,Tumor_Sample_Barcode,cDNA_Change,Codon_Change,Protein_Change,t_alt_count,t_ref_count,tumor_f,isDeleterious,isTCGAhotspot,TCGAhsCnt,isCOSMIChotspot,COSMIChsCnt,i_ExAC_AF,PASS"'
refwm.update_config(aggregate)
submission_id1 = refwm.create_submission(aggregate['name'], samplesetname)

In [ ]:
terra.waitForSubmission(refwm, [submission_id1,submission_id2])

In [ ]:
#refwm.delete_sample_set('19Q3add')

# Downloading from terra

In [ ]:
# res = refwm.get_sample_sets().loc[samplesetname]
# res

In [ ]:
# res = res['filtered_CGA_MAF_aggregated']
# ! gsutil cp $res "temp/mutation.mergedmaf.txt"

import util.postprocess_maf as pm
mutation_filtered_terra_merged = pm.download_maf_from_workspace(refwm, sample_set_ids = ['all_ice', 'all_agilent'],
                                                                output_maf='/tmp/mutation_filtered_terra_merged.txt');

# postprocessing

In [ ]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')
load('../JKBio/gkugener/DM_OMICS/Annotations.RData') 
# There are some cell lines the celllinemapr does not know how to map so we need to load this data object for now (from old datasets)

In [ ]:
%%R
source('CCLE_postp_function.R')
release <- '19Q3'

In [ ]:
%%R
library(tidyverse)
library(data.table)
library(magrittr)
library(taigr)
library(celllinemapr) # To pull out DepMap_IDs from CCLE_names where needed
newly_merged_maf <- 'temp/mutation.mergedmaf.txt'

In [ ]:
%%R
newly_merged_maf <- readMutations(newly_merged_maf)

In [ ]:
%%R
new_release <- createSNPs(newly_merged_maf)

In [ ]:
%%R
names(new_release)

### (1.5) Quick adding

Here, rather than rerunning the entire analysis, because we know we are adding only WES samples, we can download the previous release's MAF, add the samples, update any annotations, and perform any global filters at the end.

First we need to do an additional step of filtering on coverage and number 

In [ ]:
%%R
previous.release.maf <- load.from.taiga(data.name='depmap-mutations-maf-35fe',  data.version=3, data.file='mutations.19Q2')
if (colnames(previous.release.maf)[1] == 'X1' || colnames(previous.release.maf)[1] == '') {
 previous.release.maf[,1] <- NULL 
}

In [ ]:
%%R
names(previous.release.maf)

In [ ]:
%%R
merged <- addToMainMutation(previous.release.maf, new_release)

In [ ]:
%%R
names(merged)

In [ ]:
%%R
## check if some rows have nans
length(which(is.na(merged$Hugo_Symbol)))

In [ ]:
%%R
dim(merged)

In [ ]:
%%R
filtered <- filterAllelicFraction(merged)

In [ ]:
%%R
filtered <- filterMinCoverage(filtered$merged, filtered$removed_from_maf)

In [ ]:
%%R
merged %<>% dplyr::rename(Tumor_Seq_Allele2=Tumor_Seq_Allele1)

In [ ]:
%%R
clean_annotations <- mergeAnnotations(merged,previous.release.maf)

In [ ]:
%%R
new_release <- addAnnotation(filtered$merged, clean_annotations, colnames(previous.release.maf))

In [ ]:
%%R
# Save the ready to upload file to upload to taiga
write.table(
  new_release, 
  paste0('temp/mutations.', release, '.tsv'), sep = '\t', quote = F, row.names = F)

# Validation

#### Compare to previous release

I would run some checks here comparing the results to the previous releases MAF. Namely:

- Count the total number of mutations per cell line, split by type (SNP, INS, DEL)
- Count the total number of mutations observed by position (group by chromosome, start position, end position and count the number of mutations)
- Look at specific differences between the two MAFs (join on DepMap_ID, Chromosome, Start position, End position, Variant_Type). I would do this for WES only

In [ ]:
mutations = pd.read_csv('temp/mutations.'+release+'.tsv', sep='\t')

## Remove less used 

In [ ]:
len(mutations.columns)

In [ ]:
mutations[mutations["Hugo_Symbol"]=="ACOT4"][mutations['Start_position']==74058831].values

In [ ]:
ac_data = mutations[[val for val in mutations.columns.values if '_AC' in val]]

In [ ]:
ac_names = ac_data.columns.values
ac_data = ac_data.values

In [ ]:
ac_data.shape[0]

In [ ]:
allnan

In [ ]:
toofew = 0
allnan = 0
for pos, val in enumerate(ac_data):
    i = 0
    print(100*pos/ac_data.shape[0],end='\r')
    for p, v in enumerate(val):
        if v is np.nan:
            i+=1
    if i==7:
        mutations = mutations.drop[pos]
        allnan+=1

### basic counts

In [ ]:
#Count the total number of mutations per cell line, split by type (SNP, INS, DEL)


In [ ]:
# Count the total number of mutations observed by position

Are mutation consistent?

In [ ]:
#  to check this, if you group all the mutations in the mutations table by Chromosome, Start_position, End_position, Reference_Allele, Tumor_Seq_Allele1 columns, they should all have the same annotation for the other columns (protein change, exac_af, etc...)


QC mutations, for a known dependency, check if it matches mutation of this gene. (if P53 is mutated, cannot have dependency on P53 or MDM2 MDM4/ inverse fir BRAF and KRAF to themselves)

# uploading on taiga

In [ ]:
! ls temp

In [ ]:
tc.update_dataset(dataset_permaname="depmap-mutations-maf-35fe",
                 upload_file_path_dict={'temp/mutations.'+release+'.tsv': 'TableTSV'},
                 dataset_description="# Updating to "+release+ """
                 
                 reparing the missing column problem
                 
                 """)

## comparing two pipelines on same sampleset